# Image Classification with the MNIST dataset

### Learning about the MNIST handwritten digits dataset from Nvidia Deep Learning Institute. Course: Getting started with Deep Learning

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam

In [2]:
import torchvision
import torchvision.transforms.v2 as transforms
import torchvision.transforms.functional as F
import matplotlib.pyplot as plt

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.is_available()

False

### Loading the data

In [6]:
train_set = torchvision.datasets.MNIST(root='./data', train=True, download=True)
test_set = torchvision.datasets.MNIST(root='./data', train=False, download=True)

100%|██████████| 9.91M/9.91M [00:01<00:00, 7.72MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 176kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 2.25MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 10.4MB/s]


In [7]:
train_set

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train

In [9]:
test_set

Dataset MNIST
    Number of datapoints: 10000
    Root location: ./data
    Split: Test

In [10]:
x_0, y_0 = train_set[0]

In [11]:
x_0

In [12]:
y_0

5

### Tensors

In [13]:
trans = transforms.Compose([transforms.ToTensor()])
x_0_tensor = trans(x_0)

/usr/local/lib/python3.11/dist-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [14]:
x_0_tensor.dtype

torch.float32

In [15]:
x_0_tensor.min()

tensor(0.)

In [16]:
x_0_tensor.max()

tensor(1.)

In [17]:
x_0_tensor.size()

torch.Size([1, 28, 28])

In [18]:
x_0_tensor.device

device(type='cpu')

In [19]:
x_0_gpu = x_0_tensor.cuda()
x_0_gpu.device

device(type='cuda', index=0)

In [20]:
x_0_tensor.to(device).device

device(type='cuda', index=0)

In [21]:
image = F.to_pil_image(x_0_tensor)
image

### Preparaing data for training

In [22]:
trans = transforms.Compose([transforms.ToTensor()])

/usr/local/lib/python3.11/dist-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [23]:
train_set.transform = trans
test_set.transform = trans

In [24]:
batch_size = 32

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size)

### creating the model

In [25]:
layers = []
layers

[]

In [26]:
test_matrix = torch.tensor(
    [[1,2,3],
     [4,5,6],
     [7,8,9]]
)
test_matrix

tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])

In [28]:
nn.Flatten()(test_matrix)

tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])

In [29]:
batch_test_matrix = test_matrix[None, :]
batch_test_matrix

tensor([[[1, 2, 3],
         [4, 5, 6],
         [7, 8, 9]]])

In [30]:
nn.Flatten()(batch_test_matrix)

tensor([[1, 2, 3, 4, 5, 6, 7, 8, 9]])

In [31]:
nn.Flatten()(test_matrix[:, None])

tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])

In [32]:
layers = [
    nn.Flatten()
]
layers

[Flatten(start_dim=1, end_dim=-1)]

In [33]:
input_size = 1 * 28 * 28

In [34]:
# input layer

layers = [
    nn.Flatten(),
    nn.Linear(input_size, 512),  # Input
    nn.ReLU(),  # Activation for input
]
layers

[Flatten(start_dim=1, end_dim=-1),
 Linear(in_features=784, out_features=512, bias=True),
 ReLU()]

In [35]:
# hidden layer

layers = [
    nn.Flatten(),
    nn.Linear(input_size, 512),  # Input
    nn.ReLU(),  # Activation for input
    nn.Linear(512, 512),  # Hidden
    nn.ReLU()  # Activation for hidden
]
layers

[Flatten(start_dim=1, end_dim=-1),
 Linear(in_features=784, out_features=512, bias=True),
 ReLU(),
 Linear(in_features=512, out_features=512, bias=True),
 ReLU()]

In [36]:
# output layer

n_classes = 10

layers = [
    nn.Flatten(),
    nn.Linear(input_size, 512),  # Input
    nn.ReLU(),  # Activation for input
    nn.Linear(512, 512),  # Hidden
    nn.ReLU(),  # Activation for hidden
    nn.Linear(512, n_classes)  # Output
]
layers

[Flatten(start_dim=1, end_dim=-1),
 Linear(in_features=784, out_features=512, bias=True),
 ReLU(),
 Linear(in_features=512, out_features=512, bias=True),
 ReLU(),
 Linear(in_features=512, out_features=10, bias=True)]

In [37]:
model = nn.Sequential(*layers)
model

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=512, bias=True)
  (2): ReLU()
  (3): Linear(in_features=512, out_features=512, bias=True)
  (4): ReLU()
  (5): Linear(in_features=512, out_features=10, bias=True)
)

In [38]:
model.to(device)

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=512, bias=True)
  (2): ReLU()
  (3): Linear(in_features=512, out_features=512, bias=True)
  (4): ReLU()
  (5): Linear(in_features=512, out_features=10, bias=True)
)

In [39]:
next(model.parameters()).device

device(type='cuda', index=0)

In [40]:
model = torch.compile(model)

### Training the model

In [41]:
loss_function = nn.CrossEntropyLoss()

In [42]:
optimizer = Adam(model.parameters())

In [43]:
# calculating accuracy

train_N = len(train_loader.dataset)
test_N = len(test_loader.dataset)

In [44]:
def get_batch_accuracy(output, y, N):
    pred = output.argmax(dim=1, keepdim=True)
    correct = pred.eq(y.view_as(pred)).sum().item()
    return correct / N

In [48]:
# train function

def train():
    loss = 0
    accuracy = 0

    model.train()
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        output = model(x)
        optimizer.zero_grad()
        batch_loss = loss_function(output, y)
        batch_loss.backward()
        optimizer.step()

        loss += batch_loss.item()
        accuracy += get_batch_accuracy(output, y, train_N)
    print('Train - Loss: {:.4f} Accuracy: {:.4f}'.format(loss, accuracy))

In [51]:
# validate function

def validate():
    loss = 0
    accuracy = 0

    model.eval()
    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            output = model(x)

            loss += loss_function(output, y).item()
            accuracy += get_batch_accuracy(output, y, test_N)
    print('Valid - Loss: {:.4f} Accuracy: {:.4f}'.format(loss, accuracy))

In [52]:
epochs = 5

for epoch in range(epochs):
    print('Epoch: {}'.format(epoch))
    train()
    validate()

Epoch: 0
Train - Loss: 121.9849 Accuracy: 0.9801
Valid - Loss: 28.0135 Accuracy: 0.9751
Epoch: 1
Train - Loss: 86.1822 Accuracy: 0.9854
Valid - Loss: 25.0109 Accuracy: 0.9778
Epoch: 2
Train - Loss: 70.2533 Accuracy: 0.9877
Valid - Loss: 27.5048 Accuracy: 0.9754
Epoch: 3
Train - Loss: 62.4912 Accuracy: 0.9897
Valid - Loss: 26.3285 Accuracy: 0.9784
Epoch: 4
Train - Loss: 51.4802 Accuracy: 0.9913
Valid - Loss: 29.5644 Accuracy: 0.9763


In [53]:
prediction = model(x_0_gpu)
prediction

tensor([[-33.9353, -10.8709, -17.6301,  11.2807, -30.3892,  16.9676, -29.3772,
         -13.4091, -17.2117, -11.0587]], device='cuda:0',
       grad_fn=<CompiledFunctionBackward>)

In [54]:
prediction.argmax(dim=1, keepdim=True)

tensor([[5]], device='cuda:0')

In [55]:
y_0

5